### Libraries & Modules

In [1]:
# ----- BOX -----

from boxsdk import OAuth2, Client

# ----- BOX Developer Console -----

from BOX_Developer_Console import client_id, client_secret

# ----- Copy -----

from copy import copy

# ----- DateTime -----

from datetime import date

# ----- Numpy, PanDas, Re -----

import numpy
import pandas

# ----- OpenPyXl -----

from openpyxl import load_workbook, Workbook
from openpyxl.styles.alignment import Alignment
from openpyxl.styles.borders import Border, Side, BORDER_MEDIUM, BORDER_THIN
from openpyxl.styles import colors, Font
from openpyxl.styles.fills import PatternFill
from openpyxl.styles.numbers import FORMAT_NUMBER_00
from openpyxl.styles.protection import Protection
from openpyxl.utils.cell import column_index_from_string, get_column_letter

# ----- OS -----

import os

# ----- Pillow -----

from PIL import Image, ImageTk

# ----- Re -----

import re

# ----- Selenium -----

from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

# ----- TKInter -----

import tkinter
import tkinter.font
from tkinter.messagebox import showinfo
from tkinter import ttk
from ttkbootstrap import Style

### BOX Application

In [2]:
# ----- Interacting with BOX -----

class BOXApplication(Client):
    
    def __init__(self, OAuth2):
        super().__init__(OAuth2)
    
    def __str__(self):
        return f'{super().__str__()}'
    
    ## ----- File ID -----
    
    def get_file_id(self, file_name, folder_id):
        
        file_id = None
        
        file_name_with_extension = file_name.split('.')
        file_name = file_name_with_extension[0]
        file_extension = file_name_with_extension[1]
        
        my_list = self.search().query(query = f'"{file_name}"', extension = [f'{file_extension}'], limit = 1)
        
        for i in my_list:
            file_id = i.id
        if not file_id:
            raise Exception(f'File was not found in Folder ID: {folder_id}.')
        return file_id
    
    ## ----- Shared Link -----
    
    def create_shared_link(self, file_id):
        
        my_shared_link = None
        
        try:
            my_shared_link = self.file(file_id).get_shared_link(access = 'open', allow_download = True)
            print(f"Shared Link was successfully created for File ID '{file_id}'.")
            return my_shared_link
    
        except BoxAPIException as Error:
            raise Exception(f"An error ocurred while trying to create the Shared Link for File ID: '{file_id}'.")
            
    def get_shared_link(self, file_id):
        
        my_shared_link = None
    
        try:
            my_shared_link = self.file(file_id).get().shared_link
            print(f"The Shared Link for File ID '{file_id}' is the following: ")
            return my_shared_link['download_url']
    
        except BoxAPIException as Error:
            raise Exception(f"An error ocurred while trying to obtain the Shared Link for File ID '{file_id}'.")
    
    ## ----- Upload & Download -----
    
    def upload_file(self, folder_id, file_path):
        
        file_for_upload = None

        try:
            file_for_upload = self.folder(folder_id).upload(file_path)
            print(f"File was successfully uploaded to BOX Folder '{folder_id}' with the following File ID '{file_for_upload.id}'.")
    
        except BoxAPIException as Error:
            raise Exception(f"An error ocurred while trying to upload the file to BOX Folder '{folder_id}'.")
        
    def download_file(self, file_id, file_path):
    
        download_path = None
    
        try:
            download_path = open(file_path, 'wb')
            self.file(file_id).download_to(download_path)
            print(f"File ID '{file_id}' was successfully downloaded.")
        
        except BoxAPIException as Error:
            raise Exception(f"An error ocurred while trying to download File ID '{file_id}'.")
    
    # ----- Update -----
    
    def update_file(self, file_id, file_path):
        
        file_for_upload = None
        
        try:
            file_for_upload = self.file(file_id).update_contents(file_path)
            print(f"A new version was successfully uploaded for File ID '{file_id}'.")
        
        except BoxAPIException as Error:
            raise Exception(f"An error ocurred while trying to upload a new version of File ID '{file_id}'.")

In [3]:
authorization = OAuth2(client_id = client_id, 
                       client_secret = client_secret)

box_application = BOXApplication(authorization)

### Selenium & TKInter

In [4]:
# ----- Interacting with Selenium -----

chrome = webdriver.Chrome(r"C:\Users\046673613\Desktop\ODA112 - Audit Backup Response - PwC Connect\Chrome Driver\chromedriver.exe")

chrome.get('https://account.box.com/api/oauth2/authorize?response_type=code&client_id=' + client_id)

# ----- Screen Resolution -----

try:
    from ctypes import windll
    windll.shcore.SetProcessDpiAwareness(1)
except:
    pass

# ----- Interacting with TKInter -----

root = tkinter.Tk()
root.title("Confirmation Window")

frame = ttk.Frame(root, padding = (30, 15))
frame.pack()

# ----- Style & Theme -----

style_dictionary = {'Light': ['cosmo', 'flatly', 'journal', 'litera', 'lumen', 'minty', 'pulse', 'sandstone', 'united', 'yeti'], 
                    'Dark': ['darkly', 'cyborg', 'superhero', 'solar']}
style = Style(theme = 'darkly')

# ----- Functions, Variables -----

access_code = tkinter.StringVar()

def obtain_url():
    
    # Part 1: Authentication URL
    
    url = chrome.current_url
    search = re.search('code=', url)
    beggining = search.span()
    access_code = url[beggining[1]:]
    
    # Part 2: Access Token, Refresh Token
    
    access_token, refresh_token = authorization.authenticate(access_code)
    print(f'Access Code: {access_code}')
    print(f'Access Token: {access_token}, Refresh Token: {refresh_token}')
    
    # Part 3: Confirmation Message
    
    window = tkinter.Toplevel()
    
    window_label = tkinter.Label(window, text = 'Connection succeeded!')
    ok_button = tkinter.Button(window, text = "OK", command = window.destroy)
    
    window_label.pack(fill = 'x', padx = 50, pady = 5)
    ok_button.pack(fill = 'x')

# ----- Widgets -----

image = Image.open("Grant Access BOX.png").resize((750, 600))
photo = ImageTk.PhotoImage(image)
image_label = ttk.Label(frame, 
                        text = f"After granting access to the application, click the 'Authorize' button.", 
                        image = photo, 
                        padding = (50,5), 
                        compound = "bottom")

authorize_button = ttk.Button(frame, text = 'Authorize', padding = (50, 5), command = obtain_url)
quit_button = ttk.Button(frame, text = 'Quit', padding = (50, 5), command = root.destroy)

# ----- Layout -----

image_label.config(font = ('Segoe', 15))
image_label.pack(fill = 'both', expand = True)

authorize_button.pack(side = 'left', fill = 'x', expand = True)
quit_button.pack(side = 'left', fill = 'x', expand = True)

root.mainloop()

chrome.close()

Access Code: LzEThEDR2cETmUrJZFjiSRphBEI7aj1v
Access Token: GYYc8ZvkNIOODQePe8iMhHhctMrFfngL, Refresh Token: YfV8hZA0pRx1bDoDlt32pKALyUop5LEX51awXgK2QZIydZiFTaVb61mCiewUgg99


### CAS Report - Update BOX Hyperlink

In [26]:
list_of_entity_file_id = box_application.get_file_id('UAT Testing File 2.xlsx', '150340348409')
list_of_entity_file_id

'886186913452'

In [27]:
box_application.download_file(list_of_entity_file_id, 
                              os.environ['USERPROFILE'] + \
                              r'/Desktop/ODA112 - Audit Backup Response - PwC Connect/Input Files/List of Entity - Batch 1.xlsx')

File ID '886186913452' was successfully downloaded.


In [29]:
list_of_entity = pandas.read_excel(os.environ['USERPROFILE'] + \
                                   r'/Desktop/ODA112 - Audit Backup Response - PwC Connect/Input Files/List of Entity - Batch 1.xlsx', 
                                   sheet_name = 'Audit 2019', 
                                   header = 0)
list_of_entity.head(3)

,Entity Name as per PwC Connect,Entity Name as per IBM BOX,Entity backup BOX Link
0,IBM (China) Company Limited,IBM (China) Company Limited,https://ibm.ent.box.com/folder/144047211583
1,IBM - General CAS & Info,IBM - General CAS & Info,https://ibm.ent.box.com/folder/144047782412
2,IBM Capital Singapore Pte Ltd FY2019,IBM Capital Singapore Pte Ltd FY2019,https://ibm.ent.box.com/folder/144048387131


In [30]:
print(f'{list_of_entity.isnull().sum()} \n')

list_of_entity = list_of_entity.dropna()

print(f'{list_of_entity.isnull().sum()}')

Entity Name as per PwC Connect    0
Entity Name as per IBM BOX        0
Entity backup BOX Link            0
dtype: int64 

Entity Name as per PwC Connect    0
Entity Name as per IBM BOX        0
Entity backup BOX Link            0
dtype: int64


In [31]:
def get_backup_box_link(dataframe, serie):
    
    backup_box_link = []
    
    try:      
        for i in dataframe.loc[:, serie]:
            j = i.split('/')[4]
            if len(j) <= 12:
                backup_box_link.append(j)
        return backup_box_link
    
    except:
        raise Exception(f"An error ocurred while trying to parse through the DataFrame '{dataframe}'.")

In [32]:
backup_box_link = get_backup_box_link(list_of_entity, 'Entity backup BOX Link')
backup_box_link

['144047211583',
 '144047782412',
 '144048387131',
 '144045086669',
 '144046631188',
 '144047648257',
 '144046572184',
 '144045190499',
 '144046998183',
 '144047248288']

In [33]:
def get_cas_report(list):
    
    cas_report = {}
    
    try:
        for i in list:
            coincidence = box_application.folder(i).get_items()
            for j in coincidence:
                if j.name.find('.xls') != -1 or j.name.find('.xlsx') != -1:
                    cas_report.update({j.name + '__' + str(i): j.id})
        return cas_report
    
    except:
        raise Exception(f"An error ocurred while trying to iterate.")

In [34]:
cas_report_dictionary = get_cas_report(backup_box_link)
cas_report_dictionary

{'IBMC FY2019 Connect Request Details.xlsx__144047211583': '850266702178',
 'IBM - General CAS & Info - Connect Request Details - 04.Jan.2021.xlsx__144047782412': '850268283212',
 'IBM Capital Singapore Pte Ltd FY2019 Connect Request Details - 26.Jan.2021.xlsx__144048387131': '850266363445',
 'Connect Request Details - 05.Jan.2021.xlsx__144045086669': '850250642669',
 'IBM Solutions Delivery, Incorporated Connect Request Details - 26.Jan.2021.xlsx__144046631188': '850268788701',
 'Connect Request Details - 11.Sep.2020.xlsx__144047648257': '850267947457',
 'IBM World Trade Corporation - Brunei Darussalam Operations FY2019 - Connect Request Details - 06.Aug.2020.xlsx__144046572184': '850265237268',
 'SoftLayer Technologies Hong Kong Limited FY2019 - Connect Request Details - 04.Jan.2021.xlsx__144047248288': '850263864413'}

In [35]:
def transform_cas_report(dictionary):

    dictionary_key = []
    dictionary_value = []
    root_folder = []
    
    try:
        for i in list(dictionary.keys()):
            substring = i.split('__')[0]
            dictionary_key.append(substring)

        for j in dictionary.values():
            dictionary_value.append(j)
        
        for k in list(dictionary.keys()):
            substring = k.split('__')[1]
            root_folder.append(substring)
    
        cas_report = pandas.DataFrame({'File Name': dictionary_key, 
                                       'File ID': dictionary_value, 
                                       'Root Folder': root_folder})
        return cas_report
    
    except:
        raise Exception(f"An error ocurred while trying to iterate.")

In [36]:
cas_report = transform_cas_report(cas_report_dictionary)
cas_report.head(3)

,File Name,File ID,Root Folder
0,IBMC FY2019 Connect Request Details.xlsx,850266702178,144047211583
1,IBM - General CAS & Info - Connect Request Det...,850268283212,144047782412
2,IBM Capital Singapore Pte Ltd FY2019 Connect R...,850266363445,144048387131


In [37]:
for i in cas_report.index:
    box_application.download_file(cas_report['File ID'][i], os.environ['USERPROFILE'] + \
                                  r'/Desktop/ODA112 - Audit Backup Response - PwC Connect/Input Files/' + \
                                  str(i) + ' - ' + str(cas_report['File Name'][i]))

File ID '850266702178' was successfully downloaded.
File ID '850268283212' was successfully downloaded.
File ID '850266363445' was successfully downloaded.
File ID '850250642669' was successfully downloaded.
File ID '850268788701' was successfully downloaded.
File ID '850267947457' was successfully downloaded.
File ID '850265237268' was successfully downloaded.
File ID '850263864413' was successfully downloaded.


In [38]:
receptacle = []

for i in cas_report.index:
    receptacle.append(i)
    
print(f'Report Universe: {len(receptacle)}')

Report Universe: 8


In [65]:
for i in range(len(receptacle)):
    receptacle[i] = pandas.read_excel(os.environ['USERPROFILE'] + \
                                      r'/Desktop/ODA112 - Audit Backup Response - PwC Connect/Input Files/' \
                                      + str(i) + ' - ' + str(cas_report['File Name'][i]), 
                                      sheet_name = 'Requests', 
                                      skiprows = 6)
    
receptacle[0].head(1)

,Engagement,Request ID,Type of testing,Interim/initial/final additional,FSLI,Request,Description,Status,Due Date,Request Creation Date,...,Secondary Requestees,"Access Restricted to PwC Admins, Owners & Requestees Only?",Reassignment Restricted?,Response Document Download Restricted?,Template Documents,Response Documents,Discussions,Matching Criterion with BOX,Hyperlink to CAS Response,Additional Column
0,Process A - Services Accounting,1.0,NaN,interim,NaN,(0684) Latest Process SOX Flowcharts -GBS,Please provide latest process SOX flowchart fo...,Accepted,2019-11-15,2019-10-21,...,Seong Wei Chua,No,No,No,NaN,NaN,Supporting documents have been posted in Type ...,1.0 - (0684) Latest Process SOX Flowcharts -GBS,NaN,NaN


In [66]:
for i in range(len(receptacle)):
    receptacle[i]['BOX Root Folder'] = cas_report['Root Folder'][i]

receptacle[0].head(1)

,Engagement,Request ID,Type of testing,Interim/initial/final additional,FSLI,Request,Description,Status,Due Date,Request Creation Date,...,"Access Restricted to PwC Admins, Owners & Requestees Only?",Reassignment Restricted?,Response Document Download Restricted?,Template Documents,Response Documents,Discussions,Matching Criterion with BOX,Hyperlink to CAS Response,Additional Column,BOX Root Folder
0,Process A - Services Accounting,1.0,NaN,interim,NaN,(0684) Latest Process SOX Flowcharts -GBS,Please provide latest process SOX flowchart fo...,Accepted,2019-11-15,2019-10-21,...,No,No,No,NaN,NaN,Supporting documents have been posted in Type ...,1.0 - (0684) Latest Process SOX Flowcharts -GBS,NaN,NaN,144047211583


In [67]:
for i in range(len(receptacle)):
    receptacle[i]['Matching Criterion with BOX - Updated'] = receptacle[i].apply(lambda x: str(x['Request ID']) + ' - ' + str(x['Request']), 
                                                                                 axis = 1)
    receptacle[i]['Matching Criterion with BOX - Finalized'] = receptacle[i]['Matching Criterion with BOX - Updated']
    
receptacle[0].head(1)

,Engagement,Request ID,Type of testing,Interim/initial/final additional,FSLI,Request,Description,Status,Due Date,Request Creation Date,...,Response Document Download Restricted?,Template Documents,Response Documents,Discussions,Matching Criterion with BOX,Hyperlink to CAS Response,Additional Column,BOX Root Folder,Matching Criterion with BOX - Updated,Matching Criterion with BOX - Finalized
0,Process A - Services Accounting,1.0,NaN,interim,NaN,(0684) Latest Process SOX Flowcharts -GBS,Please provide latest process SOX flowchart fo...,Accepted,2019-11-15,2019-10-21,...,No,NaN,NaN,Supporting documents have been posted in Type ...,1.0 - (0684) Latest Process SOX Flowcharts -GBS,NaN,NaN,144047211583,1.0 - (0684) Latest Process SOX Flowcharts -GBS,1.0 - (0684) Latest Process SOX Flowcharts -GBS


In [68]:
character = ['<', '>', ':', '"', '/', '|', '\?', '\*']
    
for i in range(len(receptacle)):
    for j in character:
        receptacle[i]['Matching Criterion with BOX - Finalized'] = \
        receptacle[i]['Matching Criterion with BOX - Finalized'].replace(j, '', regex = True)
        
receptacle[0].head(1)

,Engagement,Request ID,Type of testing,Interim/initial/final additional,FSLI,Request,Description,Status,Due Date,Request Creation Date,...,Response Document Download Restricted?,Template Documents,Response Documents,Discussions,Matching Criterion with BOX,Hyperlink to CAS Response,Additional Column,BOX Root Folder,Matching Criterion with BOX - Updated,Matching Criterion with BOX - Finalized
0,Process A - Services Accounting,1.0,NaN,interim,NaN,(0684) Latest Process SOX Flowcharts -GBS,Please provide latest process SOX flowchart fo...,Accepted,2019-11-15,2019-10-21,...,No,NaN,NaN,Supporting documents have been posted in Type ...,1.0 - (0684) Latest Process SOX Flowcharts -GBS,NaN,NaN,144047211583,1.0 - (0684) Latest Process SOX Flowcharts -GBS,1.0 - (0684) Latest Process SOX Flowcharts -GBS


In [69]:
for i in range(len(receptacle)):
    receptacle[i]['Matching Criterion with BOX - Finalized'] = receptacle[i]['Matching Criterion with BOX - Finalized'].apply(lambda x: \
                                                                                                                              x[0:54])
    
receptacle[0].head(1)

,Engagement,Request ID,Type of testing,Interim/initial/final additional,FSLI,Request,Description,Status,Due Date,Request Creation Date,...,Response Document Download Restricted?,Template Documents,Response Documents,Discussions,Matching Criterion with BOX,Hyperlink to CAS Response,Additional Column,BOX Root Folder,Matching Criterion with BOX - Updated,Matching Criterion with BOX - Finalized
0,Process A - Services Accounting,1.0,NaN,interim,NaN,(0684) Latest Process SOX Flowcharts -GBS,Please provide latest process SOX flowchart fo...,Accepted,2019-11-15,2019-10-21,...,No,NaN,NaN,Supporting documents have been posted in Type ...,1.0 - (0684) Latest Process SOX Flowcharts -GBS,NaN,NaN,144047211583,1.0 - (0684) Latest Process SOX Flowcharts -GBS,1.0 - (0684) Latest Process SOX Flowcharts -GBS


In [70]:
for i in range(len(receptacle)):
    receptacle[i]['Matching Criterion with BOX - Finalized'] = receptacle[i]['Matching Criterion with BOX - Finalized'].apply(lambda x: 
                                                                                                                             x.strip())
    
receptacle[0].head(1)

,Engagement,Request ID,Type of testing,Interim/initial/final additional,FSLI,Request,Description,Status,Due Date,Request Creation Date,...,Response Document Download Restricted?,Template Documents,Response Documents,Discussions,Matching Criterion with BOX,Hyperlink to CAS Response,Additional Column,BOX Root Folder,Matching Criterion with BOX - Updated,Matching Criterion with BOX - Finalized
0,Process A - Services Accounting,1.0,NaN,interim,NaN,(0684) Latest Process SOX Flowcharts -GBS,Please provide latest process SOX flowchart fo...,Accepted,2019-11-15,2019-10-21,...,No,NaN,NaN,Supporting documents have been posted in Type ...,1.0 - (0684) Latest Process SOX Flowcharts -GBS,NaN,NaN,144047211583,1.0 - (0684) Latest Process SOX Flowcharts -GBS,1.0 - (0684) Latest Process SOX Flowcharts -GBS


In [71]:
for i in range(len(receptacle)):
    receptacle[i]['Matching Criterion with BOX - Letter Case'] = receptacle[i]['Matching Criterion with BOX - Finalized'].apply(lambda x: x.lower())
    
receptacle[0].head(1)

,Engagement,Request ID,Type of testing,Interim/initial/final additional,FSLI,Request,Description,Status,Due Date,Request Creation Date,...,Template Documents,Response Documents,Discussions,Matching Criterion with BOX,Hyperlink to CAS Response,Additional Column,BOX Root Folder,Matching Criterion with BOX - Updated,Matching Criterion with BOX - Finalized,Matching Criterion with BOX - Letter Case
0,Process A - Services Accounting,1.0,NaN,interim,NaN,(0684) Latest Process SOX Flowcharts -GBS,Please provide latest process SOX flowchart fo...,Accepted,2019-11-15,2019-10-21,...,NaN,NaN,Supporting documents have been posted in Type ...,1.0 - (0684) Latest Process SOX Flowcharts -GBS,NaN,NaN,144047211583,1.0 - (0684) Latest Process SOX Flowcharts -GBS,1.0 - (0684) Latest Process SOX Flowcharts -GBS,1.0 - (0684) latest process sox flowcharts -gbs


In [72]:
print(receptacle[0]['Matching Criterion with BOX - Updated'][89])
print('-----VERSUS-----')
print(receptacle[0]['Matching Criterion with BOX - Finalized'][89])

100.0 - (0684) Further query for Services Accrued Cost (Account code: 209300) - Accounting
-----VERSUS-----
100.0 - (0684) Further query for Services Accrued Cost


In [73]:
print(receptacle[0]['Matching Criterion with BOX - Updated'][144])
print('-----VERSUS-----')
print(receptacle[0]['Matching Criterion with BOX - Finalized'][144])

55.0 - 19.03(b) (0684) inventory allowance/reserve accounts (STG)
-----VERSUS-----
55.0 - 19.03(b) (0684) inventory allowancereserve acco


In [74]:
def request_and_response(list):
    
    request_template_and_response_document = {}
    
    try:
        for i in list:
            box_folder_collection = box_application.folder(i).get_items()
            for j in box_folder_collection:
                if j.name.find('.xls') == -1 and j.name.find('.xlsx') == -1 and j.name.find('.zip') == -1:
                    request_template_and_response_document[j.name + ' - ' + str(i)] = j.id
        return request_template_and_response_document
    
    except:
        raise Exception(f"An error ocurred while trying to iterate.")

In [75]:
request_template_and_response_document = request_and_response(backup_box_link)
request_template_and_response_document

{'Request Template & Response Documents - 144047211583': '144048247209',
 'Request Templates and Response Documents - 144047782412': '144047783612',
 'Request templates and response documents - 144048387131': '144048433325',
 'Request Template and Response Documents - 144045086669': '144045087869',
 'Request Templates and Response Documents - 144046631188': '144047526173',
 'Request Templates and Response Documents - 144046572184': '144047273350',
 'SL SG FY19 - 144045190499': '144048359370',
 'SL AU FY19 - 144046998183': '144048383119',
 'Request Templates and Response Documents - 144047248288': '144046857145'}

In [76]:
def get_hyperlink(dictionary):
    
    hyperlink_dictionary = {}
    
    try:
        for i in range(len(dictionary)):
            box_folder_collection = box_application.folder(list(dictionary.values())[i]).get_items()
            for j in box_folder_collection:
                hyperlink_dictionary[j.name + '__' + list(dictionary.keys())[i].split(' - ')[1]] = j.id
        
        return hyperlink_dictionary
    
    except:
        raise Exception(f"An error ocurred while trying to iterate.")

In [77]:
hyperlink_dictionary = get_hyperlink(request_template_and_response_document)
hyperlink_dictionary['1 - (0684) Latest Process SOX Flowcharts__144047211583']

'144048165306'

In [78]:
def transform_hyperlink_dictionary(dictionary):

    dictionary_key = []
    dictionary_value = []
    root_folder = []
    
    try:
        for i in dictionary.keys():
            substring = i.split('__')[0]
            dictionary_key.append(substring)

        for j in dictionary.values():
            dictionary_value.append(j)
            
        for k in dictionary.keys():
            substring = k.split('__')[1]
            root_folder.append(substring)
    
        hyperlink = pandas.DataFrame({'Matching Criterion with BOX - Finalized': dictionary_key, 
                                      'BOX Root Folder': root_folder, 
                                      'BOX Folder ID': dictionary_value})
        return hyperlink
    
    except:
        raise Exception(f"An error ocurred while trying to iterate.")

In [79]:
hyperlink = transform_hyperlink_dictionary(hyperlink_dictionary)
hyperlink.head(3)

,Matching Criterion with BOX - Finalized,BOX Root Folder,BOX Folder ID
0,1 - (0684) Latest Process SOX Flowcharts,144047211583,144048165306
1,1 - (0684) Latest Process SOX Flowcharts _STG,144047211583,144047626661
2,1 - 14.4 (0684) Tax Refunds for First 9 Months...,144047211583,144048261961


In [80]:
print(f'----- NUMBER OF RECORDS -----')
print(f'Before: {len(hyperlink)}')

hyperlink.drop_duplicates(subset = ['Matching Criterion with BOX - Finalized', 'BOX Root Folder'], 
                          keep = 'first', 
                          inplace = True)

print(f'After: {len(hyperlink)}')

----- NUMBER OF RECORDS -----
Before: 1558
After: 1558


In [81]:
hyperlink['BOX Folder ID - Format'] = hyperlink['BOX Folder ID'].apply(lambda x: 'https://ibm.ent.box.com/folder/' + str(x))
hyperlink.head(3)

,Matching Criterion with BOX - Finalized,BOX Root Folder,BOX Folder ID,BOX Folder ID - Format
0,1 - (0684) Latest Process SOX Flowcharts,144047211583,144048165306,https://ibm.ent.box.com/folder/144048165306
1,1 - (0684) Latest Process SOX Flowcharts _STG,144047211583,144047626661,https://ibm.ent.box.com/folder/144047626661
2,1 - 14.4 (0684) Tax Refunds for First 9 Months...,144047211583,144048261961,https://ibm.ent.box.com/folder/144048261961


In [82]:
hyperlink['Matching Criterion with BOX - Letter Case']  = hyperlink['Matching Criterion with BOX - Finalized'].apply(lambda x: x.lower())
hyperlink[['Matching Criterion with BOX - Finalized', 
           'Matching Criterion with BOX - Letter Case', 
           'BOX Folder ID', 
           'BOX Folder ID - Format', 
           'BOX Root Folder']].head(3)

,Matching Criterion with BOX - Finalized,Matching Criterion with BOX - Letter Case,BOX Folder ID,BOX Folder ID - Format,BOX Root Folder
0,1 - (0684) Latest Process SOX Flowcharts,1 - (0684) latest process sox flowcharts,144048165306,https://ibm.ent.box.com/folder/144048165306,144047211583
1,1 - (0684) Latest Process SOX Flowcharts _STG,1 - (0684) latest process sox flowcharts _stg,144047626661,https://ibm.ent.box.com/folder/144047626661,144047211583
2,1 - 14.4 (0684) Tax Refunds for First 9 Months...,1 - 14.4 (0684) tax refunds for first 9 months...,144048261961,https://ibm.ent.box.com/folder/144048261961,144047211583


In [83]:
hyperlink.drop(['Matching Criterion with BOX - Finalized'], axis = 1, inplace = True)
hyperlink[['Matching Criterion with BOX - Letter Case', 
           'BOX Folder ID', 
           'BOX Folder ID - Format', 
           'BOX Root Folder']].head(3)

,Matching Criterion with BOX - Letter Case,BOX Folder ID,BOX Folder ID - Format,BOX Root Folder
0,1 - (0684) latest process sox flowcharts,144048165306,https://ibm.ent.box.com/folder/144048165306,144047211583
1,1 - (0684) latest process sox flowcharts _stg,144047626661,https://ibm.ent.box.com/folder/144047626661,144047211583
2,1 - 14.4 (0684) tax refunds for first 9 months...,144048261961,https://ibm.ent.box.com/folder/144048261961,144047211583


In [84]:
for i in range(len(receptacle)):
    receptacle[i] = receptacle[i].merge(hyperlink, 
                                        how = 'left', 
                                        on = ['Matching Criterion with BOX - Letter Case', 'BOX Root Folder'])
    
receptacle[0].head(1)

,Engagement,Request ID,Type of testing,Interim/initial/final additional,FSLI,Request,Description,Status,Due Date,Request Creation Date,...,Discussions,Matching Criterion with BOX,Hyperlink to CAS Response,Additional Column,BOX Root Folder,Matching Criterion with BOX - Updated,Matching Criterion with BOX - Finalized,Matching Criterion with BOX - Letter Case,BOX Folder ID,BOX Folder ID - Format
0,Process A - Services Accounting,1.0,NaN,interim,NaN,(0684) Latest Process SOX Flowcharts -GBS,Please provide latest process SOX flowchart fo...,Accepted,2019-11-15,2019-10-21,...,Supporting documents have been posted in Type ...,1.0 - (0684) Latest Process SOX Flowcharts -GBS,NaN,NaN,144047211583,1.0 - (0684) Latest Process SOX Flowcharts -GBS,1.0 - (0684) Latest Process SOX Flowcharts -GBS,1.0 - (0684) latest process sox flowcharts -gbs,NaN,NaN


In [85]:
for i in range(len(receptacle)):
    receptacle[i]['Matching Criterion with BOX'] = receptacle[i]['Matching Criterion with BOX - Finalized']
    receptacle[i]['Hyperlink to CAS Response'] = receptacle[i]['BOX Folder ID - Format']
    
    receptacle[i].drop(['Matching Criterion with BOX - Updated', 
                        'Matching Criterion with BOX - Finalized', 
                        'Matching Criterion with BOX - Letter Case', 
                        'BOX Folder ID', 
                        'BOX Folder ID - Format', 
                        'BOX Root Folder'], axis = 1, inplace = True)
    
receptacle[0].head(1)

,Engagement,Request ID,Type of testing,Interim/initial/final additional,FSLI,Request,Description,Status,Due Date,Request Creation Date,...,Secondary Requestees,"Access Restricted to PwC Admins, Owners & Requestees Only?",Reassignment Restricted?,Response Document Download Restricted?,Template Documents,Response Documents,Discussions,Matching Criterion with BOX,Hyperlink to CAS Response,Additional Column
0,Process A - Services Accounting,1.0,NaN,interim,NaN,(0684) Latest Process SOX Flowcharts -GBS,Please provide latest process SOX flowchart fo...,Accepted,2019-11-15,2019-10-21,...,Seong Wei Chua,No,No,No,NaN,NaN,Supporting documents have been posted in Type ...,1.0 - (0684) Latest Process SOX Flowcharts -GBS,NaN,NaN


In [86]:
for i in range(len(receptacle)):
    try:
        receptacle[i].rename(columns = {'Unnamed: 28': 'Additional Column'}, inplace = True)
    except:
        pass
    
receptacle[0].head(1)

,Engagement,Request ID,Type of testing,Interim/initial/final additional,FSLI,Request,Description,Status,Due Date,Request Creation Date,...,Secondary Requestees,"Access Restricted to PwC Admins, Owners & Requestees Only?",Reassignment Restricted?,Response Document Download Restricted?,Template Documents,Response Documents,Discussions,Matching Criterion with BOX,Hyperlink to CAS Response,Additional Column
0,Process A - Services Accounting,1.0,NaN,interim,NaN,(0684) Latest Process SOX Flowcharts -GBS,Please provide latest process SOX flowchart fo...,Accepted,2019-11-15,2019-10-21,...,Seong Wei Chua,No,No,No,NaN,NaN,Supporting documents have been posted in Type ...,1.0 - (0684) Latest Process SOX Flowcharts -GBS,NaN,NaN


In [87]:
for i in range(len(receptacle)):
    probability = 1 - (receptacle[i]['Hyperlink to CAS Response'].isnull().sum() / receptacle[i].shape[0])
    print(f'DataFrame {i}: {probability:.0%}')

DataFrame 0: 1%
DataFrame 1: 25%
DataFrame 2: 71%
DataFrame 3: 68%
DataFrame 4: 81%
DataFrame 5: 0%
DataFrame 6: 57%
DataFrame 7: 69%


In [88]:
date_pattern = '(?P<split>\s+-\s+)(?P<day>\d+)\.(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\.(?P<year>\d+)\.'

regular_expression = re.compile(date_pattern)
regular_expression.split(cas_report['File Name'][1])

['IBM - General CAS & Info - Connect Request Details',
 ' - ',
 '04',
 'Jan',
 '2021',
 'xlsx']

In [89]:
cas_report['File Name - Format'] = cas_report['File Name'].apply(lambda x: regular_expression.split(x)[0])
cas_report[['File Name', 
            'File Name - Format', 
            'File ID']].head(3)

,File Name,File Name - Format,File ID
0,IBMC FY2019 Connect Request Details.xlsx,IBMC FY2019 Connect Request Details.xlsx,850266702178
1,IBM - General CAS & Info - Connect Request Det...,IBM - General CAS & Info - Connect Request Det...,850268283212
2,IBM Capital Singapore Pte Ltd FY2019 Connect R...,IBM Capital Singapore Pte Ltd FY2019 Connect R...,850266363445


In [90]:
print(cas_report['File Name'][0])
print('----- VS -----')
print(cas_report['File Name - Format'][0])

IBMC FY2019 Connect Request Details.xlsx
----- VS -----
IBMC FY2019 Connect Request Details.xlsx


In [91]:
file_extension_pattern = '(?P<file_extension>.xlsx)'

regular_expression = re.compile(file_extension_pattern)
regular_expression.split(cas_report['File Name'][0])

['IBMC FY2019 Connect Request Details', '.xlsx', '']

In [92]:
cas_report['File Name - Format'] = cas_report['File Name - Format'].apply(lambda x: regular_expression.split(x)[0])
cas_report[['File Name', 
            'File Name - Format', 
            'File ID']].head(3)

,File Name,File Name - Format,File ID
0,IBMC FY2019 Connect Request Details.xlsx,IBMC FY2019 Connect Request Details,850266702178
1,IBM - General CAS & Info - Connect Request Det...,IBM - General CAS & Info - Connect Request Det...,850268283212
2,IBM Capital Singapore Pte Ltd FY2019 Connect R...,IBM Capital Singapore Pte Ltd FY2019 Connect R...,850266363445


In [93]:
print(cas_report['File Name'][0])
print('----- VS -----')
print(cas_report['File Name - Format'][0])

IBMC FY2019 Connect Request Details.xlsx
----- VS -----
IBMC FY2019 Connect Request Details


In [94]:
for i in range(len(receptacle)):
    receptacle[i].to_excel(os.environ['USERPROFILE'] + r'/Desktop/ODA112 - Audit Backup Response - PwC Connect/Output Files/' + \
                           str(i) + ' - ' + str(cas_report['File Name - Format'][i]) + '.xlsx', 
                           sheet_name = 'Requests', 
                           index = False)
    print(f"{cas_report['File Name - Format'][i]} was downloaded successfully.")

IBMC FY2019 Connect Request Details was downloaded successfully.
IBM - General CAS & Info - Connect Request Details was downloaded successfully.
IBM Capital Singapore Pte Ltd FY2019 Connect Request Details was downloaded successfully.
Connect Request Details was downloaded successfully.
IBM Solutions Delivery, Incorporated Connect Request Details was downloaded successfully.
Connect Request Details was downloaded successfully.
IBM World Trade Corporation - Brunei Darussalam Operations FY2019 - Connect Request Details was downloaded successfully.
SoftLayer Technologies Hong Kong Limited FY2019 - Connect Request Details was downloaded successfully.


### OpenPyXL

In [95]:
def format_row(title, start_cell, end_cell, start_column, end_column):
    
    try:
        worksheet_1.merge_cells(start_cell + ':' + end_cell)
        worksheet_1[start_cell].value = title
        worksheet_1[start_cell].font = Font(name = 'Arial', 
                                            size = 11, 
                                            b = True, 
                                            color = '00000000')
    
        for column in worksheet_1.iter_cols(min_row = 6, 
                                            max_row = 6, 
                                            min_col = column_index_from_string(start_column), 
                                            max_col = column_index_from_string(end_column)):
            for cell in column:
                cell.border = Border(top = Side(border_style = BORDER_MEDIUM, color = '00000000'), 
                                     bottom = Side(border_style = BORDER_MEDIUM, color = '00000000'))

            for column in worksheet_1.iter_cols(min_row = 6, 
                                                max_row = 6, 
                                                min_col = column_index_from_string(start_column), 
                                                max_col = column_index_from_string(end_column)):
                for cell in column:
                    cell.fill = PatternFill(patternType = 'solid', 
                                            fgColor = colors.Color(rgb = '00C0C0C0'))
            
        worksheet_1[start_cell].border = Border(left = Side(border_style = BORDER_MEDIUM, color = '00000000'), 
                                                top = Side(border_style = BORDER_MEDIUM, color = '00000000'), 
                                                bottom = Side(border_style = BORDER_MEDIUM, color = '00000000'))
    
        worksheet_1[end_cell].border = Border(right = Side(border_style = BORDER_MEDIUM, color = '00000000'), 
                                              top = Side(border_style = BORDER_MEDIUM, color = '00000000'), 
                                              bottom = Side(border_style = BORDER_MEDIUM, color = '00000000'))
        
    except:
        raise Exception(f'An error ocurred while trying to parse through the Workbook.')

In [96]:
for i in range(len(receptacle)):
    
    workbook = load_workbook(os.environ['USERPROFILE'] + \
                             r'/Desktop/ODA112 - Audit Backup Response - PwC Connect/Output Files/' \
                             + str(i) + ' - ' + str(cas_report['File Name - Format'][i])+ '.xlsx')
    worksheet_1 = workbook.worksheets[0]
    worksheet_1.insert_rows(0, 6)

    maximum_row = worksheet_1.max_row
    maximum_column = worksheet_1.max_column
    print(f'Workbook {i} \nMaximum Row: {maximum_row} \nMaximum Column: {maximum_column}')

    # ----- Worksheet -----

    for row in worksheet_1.iter_rows(min_row = 6, max_row = 1500, 
                                     min_col = column_index_from_string('AC'), max_col = column_index_from_string('AZ')):
        for cell in row:
            cell.fill = PatternFill(patternType = 'solid', 
                                    fgColor = colors.Color(rgb = 'FFFFFFFF'))

    for column in worksheet_1.iter_cols(min_row = 1, max_row = 5, min_col = 1, max_col = column_index_from_string('AZ')):
        for cell in column:
            cell.fill = PatternFill(patternType = 'solid', 
                                    fgColor = colors.Color(rgb = 'FFFFFFFF'))
        
    for column in worksheet_1.iter_cols(min_row = maximum_row + 1, max_row = 1500, min_col = 1, max_col = column_index_from_string('AB')):
        for cell in column:
            cell.fill = PatternFill(patternType = 'solid', 
                                    fgColor = colors.Color(rgb = 'FFFFFFFF'))

    # ----- A -----

    worksheet_1['A2'].value = f'Date: {(date.today().strftime("%B %d, %Y"))}'
    worksheet_1['A2'].font = Font(name = 'Arial', 
                                  size = 9, 
                                  b = True, 
                                  color = '00000000')

    worksheet_1['A5'].value = 'Request Details: '
    worksheet_1['A5'].font = Font(name = 'Arial', 
                                  size = 9, 
                                  b = True, 
                                  color = '00000000')        
        
    # ----- A:AB -----

    worksheet_1.merge_cells('A1:AB1')
    worksheet_1['A1'].value = cas_report['File Name - Format'][i].split('.')[0]
    worksheet_1['A1'].font = Font(name = 'Georgia', 
                                  size = 16, 
                                  b = True, 
                                  color = colors.Color(rgb = '00993300'))
    
    # ----- A:H -----

    format_row('General Details', 'A6', 'H6', 'A', 'H')

    # ----- I:P -----

    format_row('Dates & Metrics', 'I6', 'P6', 'I', 'P')

    # ----- Q:T -----

    format_row('Assignments', 'Q6', 'T6', 'Q', 'T')

    # ----- U:W -----

    format_row('Access Restrictions', 'U6', 'W6', 'U', 'W')

    # ----- X:AC -----

    format_row('Other Details', 'X6', 'AC6', 'X', 'AC')
    
    # ----- Header -----
    
    for column in worksheet_1.iter_cols(min_row = 7, max_row = 7, 
                                        min_col = column_index_from_string('A'), max_col = maximum_column):
        for cell in column:
            cell.font = Font(name = 'Arial', 
                             size = 11, 
                             color = 'FFFFFFFF')
            cell.fill = PatternFill(patternType = 'solid', 
                                    fgColor = colors.Color(rgb = '00993300'))
        
            cell.alignment = Alignment(horizontal = 'left')
            
    # ----- Cell -----
       
    for row in worksheet_1.iter_rows(min_row = 8, max_row = maximum_row, 
                                     min_col = column_index_from_string('A'), max_col = maximum_column):
        for cell in row:
            cell.font = Font(name = 'Arial', 
                             size = 11, 
                             color = '00000000')
            cell.border = Border(left = Side(border_style = BORDER_THIN, color = '00000000'), 
                                 right = Side(border_style = BORDER_THIN, color = '00000000'), 
                                 top = Side(border_style = BORDER_THIN, color = '00000000'), 
                                 bottom = Side(border_style = BORDER_THIN, color = '00000000'))
            cell.fill = PatternFill(patternType = 'solid', 
                                    fgColor = colors.Color(rgb = 'FFFFFFFF'))
            
    # ----- Border -----
            
    for row in worksheet_1.iter_rows(min_row = 7, max_row = maximum_row, 
                                     min_col = column_index_from_string('AC'), max_col = maximum_column):
        for cell in row:
            cell.border = Border(left = Side(border_style = BORDER_THIN, color = '00000000'), 
                                 right = Side(border_style = BORDER_MEDIUM, color = '00000000'), 
                                 top = Side(border_style = BORDER_THIN, color = '00000000'), 
                                 bottom = Side(border_style = BORDER_THIN, color = '00000000'))
        
    for column in worksheet_1.iter_cols(min_row = maximum_row, max_row = maximum_row, 
                                        min_col = 1, max_col = column_index_from_string('AC')):
        for cell in column:
            cell.border = Border(left = Side(border_style = BORDER_THIN, color = '00000000'), 
                                 right = Side(border_style = BORDER_THIN, color = '00000000'), 
                                 top = Side(border_style = BORDER_THIN, color = '00000000'), 
                                 bottom = Side(border_style = BORDER_MEDIUM, color = '00000000'))
        
    worksheet_1['AC' + str(maximum_row)].border = Border(left = Side(border_style = BORDER_THIN, color = '00000000'), 
                                                         right = Side(border_style = BORDER_MEDIUM, color = '00000000'), 
                                                         top = Side(border_style = BORDER_THIN, color = '00000000'), 
                                                         bottom = Side(border_style = BORDER_MEDIUM, color = '00000000'))

    workbook.save(os.environ['USERPROFILE'] + r'/Desktop/ODA112 - Audit Backup Response - PwC Connect/Output Files/' \
                  + str(i) + ' - ' + str(cas_report['File Name - Format'][i]) + '.xlsx')
    
    print(f'Workbook {i} has been successfully updated. \n')

Workbook 0 
Maximum Row: 706 
Maximum Column: 29
Workbook 0 has been successfully updated. 

Workbook 1 
Maximum Row: 35 
Maximum Column: 29
Workbook 1 has been successfully updated. 

Workbook 2 
Maximum Row: 213 
Maximum Column: 29
Workbook 2 has been successfully updated. 

Workbook 3 
Maximum Row: 69 
Maximum Column: 29
Workbook 3 has been successfully updated. 

Workbook 4 
Maximum Row: 167 
Maximum Column: 29
Workbook 4 has been successfully updated. 

Workbook 5 
Maximum Row: 627 
Maximum Column: 29
Workbook 5 has been successfully updated. 

Workbook 6 
Maximum Row: 396 
Maximum Column: 28
Workbook 6 has been successfully updated. 

Workbook 7 
Maximum Row: 105 
Maximum Column: 29
Workbook 7 has been successfully updated. 



### CAS Report - Upload to BOX Folder

In [88]:
cas_report.head(3)

,File Name,File ID,Root Folder,File Name - Format
0,IBMC FY2019 Connect Request Details.xlsx,850266702178,144047211583,IBMC FY2019 Connect Request Details
1,IBM - General CAS & Info - Connect Request Det...,850268283212,144047782412,IBM - General CAS & Info - Connect Request Det...
2,IBM Capital Singapore Pte Ltd FY2019 Connect R...,850266363445,144048387131,IBM Capital Singapore Pte Ltd FY2019 Connect R...


In [93]:
for i in cas_report.index:
    box_application.update_file(cas_report['File ID'][i], 
                                os.environ['USERPROFILE'] + \
                                r'/Desktop/ODA112 - Audit Backup Response - PwC Connect/Output Files/' + \
                                str(i) + ' - ' + str(cas_report['File Name - Format'][i]) + '.xlsx')

A new version was successfully uploaded for File ID '850266702178'.
A new version was successfully uploaded for File ID '850268283212'.
A new version was successfully uploaded for File ID '850266363445'.
A new version was successfully uploaded for File ID '850250642669'.
A new version was successfully uploaded for File ID '850268788701'.
A new version was successfully uploaded for File ID '850267947457'.
A new version was successfully uploaded for File ID '850265237268'.
A new version was successfully uploaded for File ID '850263864413'.


### Application Enhacement